# Data Preparation

This notebook contains all steps and decisions made in the data preparation phase for the Austin Crime project.

## The Required Imports

Here we'll import all the required modules for this notebook.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from acquire import get_crime_data
import prepare

## Acquire the Data

We'll acquire the data using the get_crime_data function from the acquire module. Here we'll explicitly read from the source using an API, but going forward we will use the cache file 'Crime_Reports.csv'.

In [2]:
# Acquire the data using the API

df = get_crime_data(use_cache = False)
df.shape

Acquiring data from api


(500000, 31)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 31 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   incident_report_number       500000 non-null  object
 1   crime_type                   500000 non-null  object
 2   ucr_code                     500000 non-null  object
 3   family_violence              500000 non-null  object
 4   occ_date_time                500000 non-null  object
 5   occ_date                     500000 non-null  object
 6   occ_time                     500000 non-null  object
 7   rep_date_time                500000 non-null  object
 8   rep_date                     500000 non-null  object
 9   rep_time                     500000 non-null  object
 10  location_type                498329 non-null  object
 11  address                      500000 non-null  object
 12  zip_code                     497123 non-null  object
 13  council_distri

## Limit Time Frame of the Data

We are only interested in crimes reported between the years 2018 and 2021. Here we'll remove all observations that occur outside of this time frame.

In [4]:
# Let's see how the date information is stored in the dataframe.

df.head(1).occ_date

0    2022-05-21T00:00:00.000
Name: occ_date, dtype: object

In [5]:
# Set the occ_date column to a datetime type.

df.occ_date = pd.to_datetime(df.occ_date, format = '%Y-%m-%d')

In [6]:
df.occ_date.head()

0   2022-05-21
1   2022-05-21
2   2022-05-21
3   2022-05-21
4   2022-05-21
Name: occ_date, dtype: datetime64[ns]

In [7]:
# Subset the data to include observations between 2018-01-01 and 2021-12-31.

df = df[(df.occ_date >= '2018-01-01') & (df.occ_date <= '2021-12-31')]
df.shape

(401955, 31)

## Missing Values

Let's now investigate the missing values in our dataset and determine the best course of action for handling them.

### Summarize Null Values

In [8]:
prepare.attribute_nulls(df)

rows_missing  percent_missing
incident_report_number                  0         0.000000
crime_type                              0         0.000000
ucr_code                                0         0.000000
family_violence                         0         0.000000
occ_date_time                           0         0.000000
occ_date                                0         0.000000
occ_time                                0         0.000000
rep_date_time                           0         0.000000
rep_date                                0         0.000000
rep_time                                0         0.000000
location_type                        1094         0.002722
address                                 0         0.000000
zip_code                             2355         0.005859
council_district                     4186         0.010414
sector                               1365         0.003396
district                             1450         0.003607
pra                                  1860         0.004627
census_tract                         3404         0.008469
ucr_category                       246291         0.612733
category_description               246291         0.612733
x_coordinate                         1756         0.004369
y_coordinate                         1756         0.004369
latitude                             7170         0.017838
longitude                            7170         0.017838
location                             7170         0.017838
:@computed_region_a3it_2a2z          7173         0.017845
:@computed_region_8spj_utxs          8926         0.022206
:@computed_region_q9nd_rr82          8088         0.020122
:@computed_region_qwte_z96m        181745         0.452153
clearance_status                    45236         0.112540
clearance_date                      45246         0.112565

### ucr_category and category_description

The ucr_category and category_description columns have the most missing values. Let's investigate these columns.

In [9]:
df[['crime_type', 'ucr_code', 'ucr_category', 'category_description']].head()

crime_type ucr_code ucr_category  \
32703              THEFT FROM PERSON      610          23A   
32704                          THEFT      600          23H   
32705      ASSAULT ON PUBLIC SERVANT      903          NaN   
32706            PUBLIC INTOXICATION     2300          NaN   
32707  DOC DISCHARGE GUN - PUB PLACE     2408          NaN   

      category_description  
32703                Theft  
32704                Theft  
32705                  NaN  
32706                  NaN  
32707                  NaN

The ucr_category and category_description columns have far too many missing values to be useful to us. Additionally, the crime_type column provides similar information so we will drop these two columns (see the Drop Columns section).

### clearance_status

In [10]:
# The target variable is missing some values as well. Let's investigate.

df.clearance_status.value_counts(dropna = False)

N      281206
C       73845
NaN     45236
O        1668
Name: clearance_status, dtype: int64

The values N, C, and O (according to the data documentation) mean a case is either closed or not closed. We cannot make a reasonable assumption for what the null values in this column might mean, but we cannot drop this column because this is our target variable. We will drop the rows missing this feature because this feature is critical to our project (see the Drop Rows section).

### computed_region columns

In [11]:
df[[
    ':@computed_region_a3it_2a2z',
    ':@computed_region_8spj_utxs',
    ':@computed_region_q9nd_rr82',
    ':@computed_region_qwte_z96m'
]].head()

:@computed_region_a3it_2a2z :@computed_region_8spj_utxs  \
32703                        2856                           9   
32704                        3256                           3   
32705                        2856                           9   
32706                        2856                           9   
32707                        3641                           4   

      :@computed_region_q9nd_rr82 :@computed_region_qwte_z96m  
32703                          10                          83  
32704                           3                         806  
32705                          10                         NaN  
32706                          10                         NaN  
32707                           9                         202

We have no idea what these columns might be, we'll drop them.

### clearance_date

In [12]:
df.clearance_date.head()

32703                        NaN
32704    2022-01-10T00:00:00.000
32705    2022-01-03T00:00:00.000
32706    2021-12-31T00:00:00.000
32707    2022-01-05T00:00:00.000
Name: clearance_date, dtype: object

This feature might be useful to us later on. It is missing roughly the same number of observations as the clearance_status column. We will drop all rows missing this column.

### location data

In [13]:
df[[
    'x_coordinate',
    'y_coordinate',
    'latitude',
    'longitude',
    'location',
    'address'
]].head()

x_coordinate y_coordinate     latitude     longitude  \
32703      3115469      3115469  30.26678659  -97.73917819   
32704      3127324      3127324  30.21526412  -97.70301942   
32705      3114083      3114083  30.26373894  -97.74365146   
32706      3115566      3115566   30.2672999  -97.73885705   
32707      3129299      3129299  30.32804875  -97.69368286   

                                                location  \
32703  {'latitude': '30.26678659', 'longitude': '-97....   
32704  {'latitude': '30.21526412', 'longitude': '-97....   
32705  {'latitude': '30.26373894', 'longitude': '-97....   
32706  {'latitude': '30.2672999', 'longitude': '-97.7...   
32707  {'latitude': '30.32804875', 'longitude': '-97....   

                             address  
32703                   403 E 6TH ST  
32704  6936 E BEN WHITE BLVD SVRD WB  
32705               111 CONGRESS AVE  
32706                   406 E 6TH ST  
32707            1202 E ST JOHNS AVE

In [14]:
# Let's see an observation of the location feature.

list(df.location.head(1))

[{'latitude': '30.26678659',
  'longitude': '-97.73917819',
  'human_address': '{"address": "", "city": "", "state": "", "zip": ""}'}]

The location feature is mostly the latitude and longitude repeated. We can drop this column. The x and y coordinate columns are likely relevant to the authors of the dataset and aren't very useful to us. We can drop these columns.

### location_type

In [15]:
df.location_type.value_counts()

RESIDENCE / HOME                                   166721
HWY / ROAD / ALLEY/ STREET/ SIDEWALK                67146
PARKING /DROP LOT/ GARAGE                           50097
OTHER / UNKNOWN                                     27524
COMMERCIAL / OFFICE BUILDING                        10583
HOTEL / MOTEL / ETC.                                 9386
DEPARTMENT / DISCOUNT STORE                          8249
RESTAURANT                                           7854
GROCERY / SUPERMARKET                                6428
CONVENIENCE STORE                                    5897
SERVICE/ GAS STATION                                 5558
DRUG STORE / DOCTOR'S OFFICE / HOSPITAL              4703
BAR / NIGHTCLUB                                      4456
PARK / PLAYGROUND                                    3630
SPECIALTY  STORE                                     3214
AIR / BUS / TRAIN TERMINAL                           2689
GOVERNMENT / PUBLIC BUILDING                         1917
CONSTRUCTION S

There aren't too many missing values in this column. There is also a value for OTHER/UNKNOWN that we can use to impute the missing values.

### zip_code and council_district

In [16]:
df[['zip_code', 'council_district']].head(20)

zip_code council_district
32703    78701                9
32704    78741                3
32705    78701                9
32706    78701                9
32707    78752                4
32708    78758                4
32709    78753                1
32710    78701                9
32711    78702                3
32712    78702                3
32713    78744                2
32714    78752                4
32715    78750                6
32716    78741                3
32717    78701                9
32718    78758                7
32719    78741                3
32720    78704                9
32721    78752                4
32722    78744                2

In [17]:
df.council_district.value_counts()

9     61916
3     57952
4     56708
7     45643
1     45579
2     38974
5     32371
6     23420
10    18452
8     16754
Name: council_district, dtype: int64

In [18]:
df.zip_code.value_counts()

78741    31784
78753    31548
78758    31188
78701    26326
78704    26025
78745    23796
78723    21892
78744    21539
78702    17367
78752    14106
78748    13930
78759    13903
78751    10369
78705     9344
78757     9218
78721     7220
78749     6960
78724     6773
78727     6514
78729     6478
78754     6128
78731     5790
78703     5602
78750     5478
78746     5243
78717     3584
78617     3497
78735     3381
78747     3295
78756     3294
78660     3038
78722     2969
78726     2605
78719     2103
78613     1955
78736     1053
78730     1014
78739      997
78725      649
78742      470
78653      354
78728      261
78712      180
78652      125
78732       72
78737       63
78733       26
78610       23
78681       20
78664       15
78738       10
78734        8
78665        6
78641        6
78640        2
78612        2
78616        1
78645        1
Name: zip_code, dtype: int64

One of our initial questions depends on the council_district feature so we can't drop this column. It is possible that rows missing zip_code are also missing council_district. We will drop rows missing zip_code and then impute the remaining missing values in council_district.

### sector, district, and pra

We want to keep the sector and district columns as these may be useful to us in our exploration. We will remove the rows missing these values. The police reporting area column on the other hand may not be useful to us so we'll drop this column.

### census_tract

In [19]:
df.census_tract.value_counts()

11        23734
21        16599
3         10632
15         9667
204        9166
          ...  
356           1
203.53        1
461           1
18.54         1
22.02         1
Name: census_tract, Length: 315, dtype: int64

We don't think this column will be useful to us. We'll drop it.

## Drop Columns

Here we'll drop all columns that are either not useful or have too many missing values to be of any use to us.

In [20]:
# These are all the columns that will be dropped from the dataframe.

columns = [
    'incident_report_number',
    'ucr_code',
    'ucr_category',
    'category_description',
    ':@computed_region_a3it_2a2z',
    ':@computed_region_8spj_utxs',
    ':@computed_region_q9nd_rr82',
    ':@computed_region_qwte_z96m',
    'x_coordinate',
    'y_coordinate',
    'location',
    'census_tract',
    'pra',
    'occ_date_time',
    'rep_date_time'
]

df = df.drop(columns = columns)
df.shape

(401955, 16)

## Drop Rows

Here we'll drop rows with missing values that cannot be reasonabled imputed with a value.

In [21]:
df.clearance_status.value_counts(dropna = False)

N      281206
C       73845
NaN     45236
O        1668
Name: clearance_status, dtype: int64

In [ ]:
columns = [
    'clearance_status',
    'clearance_date',
    'zip_code',
    'sector',
    'district',
    'latitude',
    'longitude'
]

for column in columns:
    df = df[~df[column].isna()]

In [ ]:
df.shape

(349530, 16)

In [ ]:
prepare.attribute_nulls(df)

rows_missing  percent_missing
crime_type                   0         0.000000
family_violence              0         0.000000
occ_date                     0         0.000000
occ_time                     0         0.000000
rep_date                     0         0.000000
rep_time                     0         0.000000
location_type              753         0.002154
address                      0         0.000000
zip_code                     0         0.000000
council_district          1438         0.004114
sector                       0         0.000000
district                     0         0.000000
latitude                     0         0.000000
longitude                    0         0.000000
clearance_status             0         0.000000
clearance_date               0         0.000000

## Impute Missing Values

Here we'll fill missing values for some columns with a value we have decided upon.

In [ ]:
df['location_type'] = df.location_type.fillna('OTHER / UNKNOWN')
df['council_district'] = df.council_district.fillna(9)

In [ ]:
prepare.attribute_nulls(df)

rows_missing  percent_missing
crime_type                   0              0.0
family_violence              0              0.0
occ_date                     0              0.0
occ_time                     0              0.0
rep_date                     0              0.0
rep_time                     0              0.0
location_type                0              0.0
address                      0              0.0
zip_code                     0              0.0
council_district             0              0.0
sector                       0              0.0
district                     0              0.0
latitude                     0              0.0
longitude                    0              0.0
clearance_status             0              0.0
clearance_date               0              0.0

## Rename Columns

Now for readability we will rename some of the columns to more easily understandable names.

In [ ]:
mapper = {
    'occ_date' : 'occurence_date',
    'occ_time' : 'occurence_time',
    'rep_date' : 'report_date',
    'rep_time' : 'report_time'
}

df = df.rename(columns = mapper)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 349530 entries, 32704 to 434657
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   crime_type        349530 non-null  object        
 1   family_violence   349530 non-null  object        
 2   occurence_date    349530 non-null  datetime64[ns]
 3   occurence_time    349530 non-null  object        
 4   report_date       349530 non-null  object        
 5   report_time       349530 non-null  object        
 6   location_type     349530 non-null  object        
 7   address           349530 non-null  object        
 8   zip_code          349530 non-null  object        
 9   council_district  349530 non-null  object        
 10  sector            349530 non-null  object        
 11  district          349530 non-null  object        
 12  latitude          349530 non-null  object        
 13  longitude         349530 non-null  object        
 14  

## Rename clearance_status Values

The values in the clearance_status column are rather unreadable. We will change the values to more readable values.

In [ ]:
# We'll use this map to rename the values in the clearance_status column.

mapper = {
    'N' : 'not cleared',
    'O' : 'cleared by exception',
    'C' : 'cleared by arrest'
}

df['clearance_status'] = df.clearance_status.map(mapper)

In [ ]:
df.clearance_status.value_counts()

not cleared             275534
cleared by arrest        72423
cleared by exception      1573
Name: clearance_status, dtype: int64

## Assure Data Types Are Correct

Finally, let's ensure that the data types for all our columns are correct.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 349530 entries, 32704 to 434657
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   crime_type        349530 non-null  object        
 1   family_violence   349530 non-null  object        
 2   occurence_date    349530 non-null  datetime64[ns]
 3   occurence_time    349530 non-null  object        
 4   report_date       349530 non-null  object        
 5   report_time       349530 non-null  object        
 6   location_type     349530 non-null  object        
 7   address           349530 non-null  object        
 8   zip_code          349530 non-null  object        
 9   council_district  349530 non-null  object        
 10  sector            349530 non-null  object        
 11  district          349530 non-null  object        
 12  latitude          349530 non-null  object        
 13  longitude         349530 non-null  object        
 14  

In [ ]:
df.head()

crime_type family_violence occurence_date  \
32704                          THEFT               N     2021-12-31   
32705      ASSAULT ON PUBLIC SERVANT               N     2021-12-31   
32706            PUBLIC INTOXICATION               N     2021-12-31   
32707  DOC DISCHARGE GUN - PUB PLACE               N     2021-12-31   
32708    AGG ASLT STRANGLE/SUFFOCATE               Y     2021-12-31   

      occurence_time              report_date report_time  \
32704           2350  2022-01-07T00:00:00.000        1412   
32705           2350  2021-12-31T00:00:00.000        2350   
32706           2350  2021-12-31T00:00:00.000        2350   
32707           2347  2021-12-31T00:00:00.000        2347   
32708           2340  2022-01-01T00:00:00.000          44   

                              location_type                        address  \
32704                       OTHER / UNKNOWN  6936 E BEN WHITE BLVD SVRD WB   
32705          COMMERCIAL / OFFICE BUILDING               111 CONGRESS AVE   
32706  HWY / ROAD / ALLEY/ STREET/ SIDEWALK                   406 E 6TH ST   
32707                      RESIDENCE / HOME            1202 E ST JOHNS AVE   
32708                      RESIDENCE / HOME             10000 N LAMAR BLVD   

      zip_code council_district sector district     latitude     longitude  \
32704    78741                3     HE        5  30.21526412  -97.70301942   
32705    78701                9     GE        3  30.26373894  -97.74365146   
32706    78701                9     GE        2   30.2672999  -97.73885705   
32707    78752                4     ID        1  30.32804875  -97.69368286   
32708    78758                4     ED        1  30.36926216  -97.69510501   

        clearance_status           clearance_date  
32704        not cleared  2022-01-10T00:00:00.000  
32705  cleared by arrest  2022-01-03T00:00:00.000  
32706  cleared by arrest  2021-12-31T00:00:00.000  
32707        not cleared  2022-01-05T00:00:00.000  
32708        not cleared  2022-01-05T00:00:00.000

In [32]:
# latitude and longitude are more accurately represented as numerical types.

df.latitude = df.latitude.astype('float')
df.longitude = df.longitude.astype('float')

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 349530 entries, 32704 to 434657
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   crime_type        349530 non-null  object        
 1   family_violence   349530 non-null  object        
 2   occurence_date    349530 non-null  datetime64[ns]
 3   occurence_time    349530 non-null  object        
 4   report_date       349530 non-null  object        
 5   report_time       349530 non-null  object        
 6   location_type     349530 non-null  object        
 7   address           349530 non-null  object        
 8   zip_code          349530 non-null  object        
 9   council_district  349530 non-null  object        
 10  sector            349530 non-null  object        
 11  district          349530 non-null  object        
 12  latitude          349530 non-null  float64       
 13  longitude         349530 non-null  float64       
 14  

In [34]:
df.occurence_time = df.occurence_time.apply(lambda time: f'{int(time):04d}')
df.report_time = df.report_time.apply(lambda time: f'{int(time):04d}')

In [35]:
# We want to change the date and time columns to datetime types.

df.report_date = pd.to_datetime(df.report_date, format = '%Y-%m-%d')
df.clearance_date = pd.to_datetime(df.clearance_date, format = '%Y-%m-%d')
df.occurence_time = pd.to_datetime(df.occurence_time, format = '%H%M')
df.report_time = pd.to_datetime(df.report_time, format = '%H%M')

df.occurence_time = df.occurence_time.dt.strftime('%H:%M')
df.report_time = df.report_time.dt.strftime('%H:%M')

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 349530 entries, 32704 to 434657
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   crime_type        349530 non-null  object        
 1   family_violence   349530 non-null  object        
 2   occurence_date    349530 non-null  datetime64[ns]
 3   occurence_time    349530 non-null  object        
 4   report_date       349530 non-null  datetime64[ns]
 5   report_time       349530 non-null  object        
 6   location_type     349530 non-null  object        
 7   address           349530 non-null  object        
 8   zip_code          349530 non-null  object        
 9   council_district  349530 non-null  object        
 10  sector            349530 non-null  object        
 11  district          349530 non-null  object        
 12  latitude          349530 non-null  float64       
 13  longitude         349530 non-null  float64       
 14  

In [36]:
df.head()

crime_type family_violence occurence_date  \
32704                          THEFT               N     2021-12-31   
32705      ASSAULT ON PUBLIC SERVANT               N     2021-12-31   
32706            PUBLIC INTOXICATION               N     2021-12-31   
32707  DOC DISCHARGE GUN - PUB PLACE               N     2021-12-31   
32708    AGG ASLT STRANGLE/SUFFOCATE               Y     2021-12-31   

      occurence_time report_date report_time  \
32704          23:50  2022-01-07       14:12   
32705          23:50  2021-12-31       23:50   
32706          23:50  2021-12-31       23:50   
32707          23:47  2021-12-31       23:47   
32708          23:40  2022-01-01       00:44   

                              location_type                        address  \
32704                       OTHER / UNKNOWN  6936 E BEN WHITE BLVD SVRD WB   
32705          COMMERCIAL / OFFICE BUILDING               111 CONGRESS AVE   
32706  HWY / ROAD / ALLEY/ STREET/ SIDEWALK                   406 E 6TH ST   
32707                      RESIDENCE / HOME            1202 E ST JOHNS AVE   
32708                      RESIDENCE / HOME             10000 N LAMAR BLVD   

      zip_code council_district sector district   latitude  longitude  \
32704    78741                3     HE        5  30.215264 -97.703019   
32705    78701                9     GE        3  30.263739 -97.743651   
32706    78701                9     GE        2  30.267300 -97.738857   
32707    78752                4     ID        1  30.328049 -97.693683   
32708    78758                4     ED        1  30.369262 -97.695105   

        clearance_status clearance_date  
32704        not cleared     2022-01-10  
32705  cleared by arrest     2022-01-03  
32706  cleared by arrest     2021-12-31  
32707        not cleared     2022-01-05  
32708        not cleared     2022-01-05